In [ ]:
import os
import json
import argparse
import numpy as np
import config, consts, paths
from decoding.GPT import GPT
from decoding.utils_stim import get_stim
from decoding.utils_resp import get_resp_test
from decoding.StimulusModel import LMFeatures
from encoding.ridge import ridge, bootstrap_ridge
from utils import flatten_list, save_data
import h5py

In [14]:
subject='UTS03'

In [15]:
def convert_to_serializable(downsampled_feat):
    """Convert downsampled feature dictionary to a serializable format."""
    
    serializable_dict = downsampled_feat.tolist()

    return serializable_dict

In [17]:
def get_response(stories, subject):
	"""Get the subject"s fMRI response for stories."""
	#main_path = pathlib.Path(__file__).parent.parent.resolve()
	subject_dir = os.path.join(config.DATA_DIR, "derivative/preprocessed_data/%s" % subject)
	base = subject_dir
	resp = []
	run_on_set = []
	for story in stories:
		resp_path = os.path.join(base, "%s.hf5" % story)
		hf = h5py.File(resp_path, "r")
		resp.extend(hf["data"][:])
		if not run_on_set:
			run_on_set.append(hf["data"][:].shape[0])
		else:
			run_on_set.append(run_on_set[-1]+hf["data"][:].shape[0])
		#print(hf["data"][:].shape[0], "for story:", story)
		hf.close()
	return np.array(resp), run_on_set[:-1]

In [18]:
# load gpt
stories = flatten_list(consts.STORIES)
stories_t =consts.STORIES_test
with open(os.path.join(config.DATA_LM_DIR, "perceived", "vocab.json"), "r") as f:
    gpt_vocab = json.load(f)
gpt = GPT(path = os.path.join(config.DATA_LM_DIR, "perceived", "model"), vocab = gpt_vocab)
features = LMFeatures(model = gpt, layer = config.GPT_LAYER, context_words = config.GPT_WORDS)


In [20]:
# estimate noise model
num_voxels = consts.NUM_VOXELS[subject]
rstim, tr_stats, word_stats = get_stim(stories, "story", features)
alphas = np.zeros(num_voxels)
bscorrs = np.zeros([len(config.ALPHAS), num_voxels, config.NBOOTS])
voxels = np.sort(np.argsort(bscorrs)[-config.VOXELS:])
#splits = np.array_split(range(num_voxels), 2)
weights = np.zeros([rstim.shape[1], num_voxels])

In [34]:
rstim.shape

(1869, 3072)

In [27]:
zRresp_t = get_resp_test(subject)
zRresp_t.shape

(291, 95556)

In [33]:
zRresp,run_on_set = get_response(stories, subject)
zRresp.shape

(1869, 95556)

In [16]:
run_on_set

[343, 698, 1065, 1465]

In [28]:
save_location=os.path.join(paths.EM % subject)
#os.makedirs(save_location, exist_ok=True)
#with open(save_location+'/run_on.json', "w") as file:
#    json.dump(run_on_set,file, indent=4)

In [30]:
save_location

'/home/ckj24/genevieve/Cross-participant-semantic-decoding/results/encoding/UTS03'

In [38]:
len(convert_to_serializable(rstim))

1869

In [40]:
with open(save_location+'/features_train.json', "w") as file:
        json.dump(convert_to_serializable(rstim),file, indent=4)

In [29]:
with open(save_location+'/fmri_train.json', "w") as file:
        json.dump(convert_to_serializable(zRresp),file, indent=4)
with open(save_location+'/features_train.json', "w") as file:
        json.dump(convert_to_serializable(rstim),file, indent=4)

In [ ]:
#save_location=os.path.join(paths.EM % subject)
#with open(save_location+'/fmri_test.json', "w") as file:
#        json.dump(convert_to_serializable(zRresp),file, indent=4)
#with open(save_location+'/features_test.json', "w") as file:
#        json.dump(convert_to_serializable(rstim),file, indent=4)

In [ ]:
# estimate noise model
num_voxels = consts.NUM_VOXELS[subject]
rstim, tr_stats, word_stats = get_stim(stories, "story", features)
splits = np.array_split(range(num_voxels), 2)
weights = np.zeros([rstim.shape[1], num_voxels])
alphas = np.zeros(num_voxels)
bscorrs = np.zeros([len(config.ALPHAS), num_voxels, config.NBOOTS])
for split in splits:
    rresp = get_resp(subject, stories, "story", voxels = split, stack = True)
    weights[:, split], alphas[split], bscorrs[:, split, :] = bootstrap_ridge(rstim, rresp, alphas = config.ALPHAS, 
            nboots = config.NBOOTS, chunklen = config.CHUNKLEN, use_corr = False, seed = 42)        
    del rresp
bscorrs = bscorrs.mean(2).max(0)
voxels = np.sort(np.argsort(bscorrs)[-config.VOXELS:])

In [10]:
# estimate noise model
stim_dict = {story : get_stim([story], "story", features, tr_stats = tr_stats) for story in stories}
resp_dict = get_resp(subject, stories, "story", voxels = voxels, stack = False)
noise_model = np.zeros([len(voxels), len(voxels)])
for hstory in stories:
    tstim, hstim = np.vstack([stim_dict[tstory] for tstory in stories if tstory != hstory]), stim_dict[hstory]
    tresp, hresp = np.vstack([resp_dict[tstory] for tstory in stories if tstory != hstory]), resp_dict[hstory]
    bs_weights = ridge(tstim, tresp, alphas[voxels])
    resids = hresp - hstim.dot(bs_weights)
    bs_noise_model = resids.T.dot(resids)
    noise_model += bs_noise_model / np.diag(bs_noise_model).mean() / len(stories)
del stim_dict, resp_dict

In [11]:
save_location=os.path.join(paths.EM % subject)
em = {}
em["bscorrs"] = bscorrs
em["voxels"] = voxels
em["tr_stats"] = tr_stats
em["word_stats"] = word_stats
em["stories"] = stories
em["weights"] = weights[:, voxels]
em["noise_model"] = noise_model
save_data(save_location, em)